# Tube Components Initial Stiffness Calculator for Inno3DPnPJoint Element

## Introduction

Welcome to the **Inno3DPnPJoint_component_calc** jupyter notebook!

This notebook was designed to facilitate the calculation of the initial stiffness of the tube compoenents (i.e., face and interaction components) of the Inno3DPnPJoint beam-to-column finite element implemented in OpenSees.

This notebook was created as part of the Ph.D. thesis titled "INNOVATIVE PLUG-AND-PLAY JOINTS FOR HYBRID TUBULAR CONSTRUCTIONS, authored by Cristian V. Miculaș ([cristian.miculas@uc.pt](mailto:cristian.miculas@uc.pt)), and supervised by Ricardo Joel Teixeira Costa ([rjcosta@dec.uc.pt](mailtu:rjcosta@dec.uc.pt)) and Luís Alberto Proença Simões da Silva ([luisss@dec.uc.pt](mailto:luisss@dec.uc.pt)). The work was developed at the Institute for Sustainability and Innovation in Structural Engineering (www.isise.net), Coimbra Branch, within the University of Coimbra, Portugal (www.uc.pt).

The research leading to this notebook was made possible by funding from the Portuguese Foundation for Science and Technology (Fundação para a Ciência e a Tecnologia) under grant agreement SFRH/BD/138151/2018 attributed to the author.

Thank you for using this notebook. We hope it will be helpful for your research. Also, feel free to submit changes and remember to cite the work.

References related to this notebook:
- Miculaş, C.V., Costa, R.J., Simões da Silva, L., Simões, R., Craveiro, H., Tankova, T. (2023). Macro-modelling of the Three-Dimensional Interaction Between the Faces of a Steel Tubular Column Joint. In: Di Trapani, F., Demartino, C., Marano, G.C., Monti, G. (eds) Proceedings of the 2022 Eurasian OpenSees Days. EOS 2022. Lecture Notes in Civil Engineering, vol 326. Springer, Cham. https://doi.org/10.1007/978-3-031-30125-4_37


<table><tr><td><img src="logo_ISISE.png"></td><td><img src="logo_UC.png"></td><td><img src="logo_FCT.png"></td></tr></table>

## Representation of the Tube Components

The Tube Components are represented in the figure below:

<figure id="fig_TC">
<img src="fig_TC.png" alt="Image not found." style="width: 300px;">
    <figcaption style="text-align: center"><i><b> Tube components: 25 to 28 - face components and 29 to 32 interaction components.</i></b></figcaption>
</figure>

## Install Packages

In [ ]:
!pip install numpy
!pip install watermark

## Import Packages

In [ ]:
import numpy as np
import math

## User Input Data

### Geometry

<figure id="fig_geometry_C">
<img src="fig_geometry_C.png" alt="Image not found.">
    <figcaption style="text-align: center"><i><b> Geometry-related required variables.</i></b></figcaption>
</figure>

Variables below are shown in <a href="#fig_geometry_C"> figure above</a>.

In [ ]:
# column width --- face 1 (short)
bC = 200 # [mm]

# column height --- face 2 (long)
hC = 300 # [mm]

# column thickness
t_c = 10 # [mm]

# column external radius
radE = 11.5 # [mm]

# socket face height 
hS = 120 # [mm]

# socket face thickness
tS = 10 # [mm]

# socket face distance
dS = 20 # [mm]

In [ ]:
# rigid area width
LS = dS + 2*tS; # [mm]

### Material

In [ ]:
# modulus of elasticity:
E = 210000 # [N/mm**2]

## Initial stiffness

###  - function definition

The computation of the stiffness of the tube components is based on the work of Neves and Gomes <a href="#ref1">[1]</a>, and Simões da Silva et al. <a href="#ref2">[2]</a>, referred to as the _Neves-Gomes Model (NGM)_, on the out-of-plane behavior of the column web plate for beam-to-minor-axis column joints.

In the NGM, the initial stiffness, $S_i$, is defined as:

$$S_{i} \ = \ \dfrac{16 \ E \ t_{c}^3}{L^2} \dfrac{\alpha \ + \ \left( 1 \ - \ \beta \right) \ \tan \theta}{\left( 1 \ - \ \beta \right) ^3 \ + \ \dfrac{10.4 \ \left( k_1 \ - \ k_2 \ \beta \right) }{\mu^2}}$$

The above equation is defined in this notebok as a function:

In [ ]:
def calc_Si_GNM(E, t_c, L, u, f):
    alpha = u/L
    beta  = f/L
    mu    = L/t_c

    k_1 = 1.50
    k_2 = 1.63

    theta_rad = np.deg2rad(35 - 10*beta)

    Si = (16*E*t_c**3) / L**2 * (((alpha + (1-beta) * math.tan(theta_rad))) / ((1-beta)**3 + 10.4*(k_1-k_2*beta)/mu**2))
    
    return Si

where the variables are represented below:

<figure id="fig_GNM">
<img src="fig_GNM.png" alt="Image not found." style="width: 600px;">
    <figcaption style="text-align: center"><i><b>Gomes-Neves Model.</i></b></figcaption>
</figure>

### - computation

In [ ]:
# length of face 1 (short)
L_1_NGM = bC - 2*radE # [mm]

# initial stiffness
Si_NGM_1 = calc_Si_GNM(E, t_c, L_1_NGM, hS, LS)
print("The initial stiffness of face 1 (short) is:", Si_NGM_1, "[N/mm].")

In [ ]:
# length of face 2 (long)
L_2_NGM = hC - 2*radE

# initial stiffness
Si_NGM_2 = calc_Si_GNM(E, t_c, L_2_NGM, hS, LS)
print("The initial stiffness of face 2 (long) is:", Si_NGM_2, "[N/mm].")

## Equivalent Frame Models (EFMs)

The computation of the stiffness of the tube components can be performed usign the 6 EFMs, as follows:

- _Harada frame-model for square cross-sections (**HS-EFM**)_: developed by Harada and Simões da Silva [<a href="#ref3">3</a>,<a href="#ref4">4</a>];


    
- _Harada extended frame-model for rectangular cross-sections (HS-EFM)_: an extension of HS-EFM applicable to both square and rectangular cross-sections and comprises of two sub-configurations:

    - _**HR-EFM-IF**_: to each column face is assigned its corresponding moment of inertia (IF - inertia full); thus, there are two moments of inertia $I_{1}$ (width) and $I_{2}$ (length);
    
    - _**HR-EFM-IEq**_: to each column face is assigned the mathematical average of the moments of inertia of the column faces (IEq - inertia equivalent); thus, all faces have the same moment of inertia: $I_{eq} = \dfrac{I_{1} + I_{2}}{2}$.
        
    
- _Proposed frame-model for square cross-sections (**PS-EFM**)_: based on HS-EFM but modified to account for the connection's size, a feature that makes it suitable for a larger variety of joint dimensions; thus, it is appropriate for the plug-and-play beam-to-column connection;

    
- _Proposed frame-model for rectangular cross-sections (PR-EFM)_: an extension of the PS-EFM and applies to both square and rectangular cross-sections and comprises of two sub-configurations:

    - _**PR-EFM-IF**_: to each column face is assigned its corresponding moment of inertia (IF - inertia full); thus, there are two moments of inertia $I_{1}$ (width) and $I_{2}$ (length);
    
    - _**PR-EFM-IEq**_: to each column face is assigned the mathematical average of the moments of inertia of the column faces (IEq - inertia equivalent); thus, all faces have the same moment of inertia: $I_{eq} = \dfrac{I_{1} + I_{2}}{2}$;


A generic representation of the EFMs is presented below:

<figure id="fig_EFM">
    <table>
      <tr>
        <td style="text-align: center;"><img src="fig_HR_EFM.png"></td>
        <td style="text-align: center;"><img src="fig_PR_EFM.png"></td>
      </tr>
      <tr>
        <td style="text-align: center;">Harada</td>
        <td style="text-align: center;">Proposed</td>
      </tr>
    </table>
    <figcaption style="text-align: center"><i><b>Equivalent Frame Models (EFMs).</i></b></figcaption>
</figure>

The remaining parts of the notebook are divided into two sections: (i)  on to the Harada EFMs and (ii) on to the Proposed EFMs.
    

---

## Harada EFM

If the user inputs the same value for the column width, bC, and the column height, hC (i.e., a square cross-section), the results obtained in this section are for the HS-EFM (both IF and IEq will provide identical results). Otherwise the calculation is performed for HR-EFM, and the HR-EFm-IF and HR-EFM-IEq will provide different results.

### IF: moment of inertia computation

generic formula face 1: $$ I_{1}   \ = \ \dfrac{L_{1}^3}{192 \ E} \ S_{i.1} $$

In [ ]:
# moment of inertia of face 1 (short)
I_Harada_1 = L_1_NGM**3 * Si_NGM_1 / 192 / E
print("The moment of inertia of face 1 (short) using Harada EFM is:", I_Harada_1, "[mm^4].")

generic formula face 2: $$ I_{2}   \ = \ \dfrac{L_{2}^3}{192 \ E} \ S_{i.2} $$

In [ ]:
# moment of inertia of face 2 (long)
I_Harada_2 = L_2_NGM**3 * Si_NGM_2 / 192 / E
print("The moment of inertia of face 2 (long) using Harada EFM is:", I_Harada_2, "[mm^4].")

### IF: stiffness of the tube compoenents

generic formula:

$$ k_{\text{comp}.25} \ = \ k_{\text{comp}.27} \ = \ \dfrac{48 \  E \  I_{1} \ \left( - \ 3 \  I_{2} \  L_{1}^{2} \ + \ 4 \  I_{2} \ L_{1} \ L_{2} \ + \ I_{1} \ L_{2}^{2} \right) }{L_{1}^{3} \ L_{2} \ \left( I_{1} \ L_{2} \ + \ I_{2} \ L_{1} \right)} $$

$$ 
k_{\text{comp}.26} \ = \ k_{\text{comp}.28}  \ = \ \dfrac{48 \ E \ I_{2} \ \left(I_{2} \ L_{1}^{2} \ + \ 4 \ I_{1} \ L_{1} \ L_{2} \ - \ 3 \ I_{1} \ L_{2}^{2} \right) }{L_{1} \ L_{2}^{3} \ \left(I_{1} \ L_{2} \ + \ I_{2} \ L_{1} \right)} $$


$$ k_{\text{comp}.29} \ = \ k_{\text{comp}.30} \ = \ k_{\text{comp}.31} \ = \ k_{\text{comp}.32} \ = \ \dfrac{72 \ E \ I_{1} \ I_{2}}{L_{1} \ L_{2} \ \left( I_{1} \ L_{2} \ + \ I_{2} \ L_{1} \right) } $$

In [ ]:
# component 25
k_comp_25_Harada = (48*E*I_Harada_1*(- 3*I_Harada_2*L_1_NGM**2 + 4*I_Harada_2*L_1_NGM*L_2_NGM +   I_Harada_1*L_2_NGM**2))/(L_1_NGM**3*L_2_NGM*(I_Harada_1*L_2_NGM + I_Harada_2*L_1_NGM));

# component 26
k_comp_26_Harada = (48*E*I_Harada_2*(    I_Harada_2*L_1_NGM**2 + 4*I_Harada_1*L_1_NGM*L_2_NGM - 3*I_Harada_1*L_2_NGM**2))/(L_1_NGM*L_2_NGM**3*(I_Harada_1*L_2_NGM + I_Harada_2*L_1_NGM));

# component 29
k_comp_29_Harada = (72*E*I_Harada_1*I_Harada_2)/(L_1_NGM*L_2_NGM*(I_Harada_1*L_2_NGM + I_Harada_2*L_1_NGM));

# print
print("The moment of the tube components using Harada EFM-IF is:\n")
print("components 25 and 27 (face component):", k_comp_25_Harada, "[N/mm].")
print("components 26 and 28 (face component):", k_comp_26_Harada, "[N/mm].")
print("components 29, 30, 31 and 32 (interaction component):", k_comp_29_Harada, "[N/mm].")

### IEq: moment of inertia computation

generic formula:
$$I_{eq} = \dfrac{I_{1} + I_{2}}{2}$$

In [ ]:
# equivalent moment of inertia
I_Harada_eq = (I_Harada_1 + I_Harada_2) / 2
print("The equivalent moment of inertia using Harada EFM is:", I_Harada_eq, "[mm^4].")

### IEq: stiffness of the tube compoenents

generic formula:

$$ k_{\text{comp}.25} \ = \ k_{\text{comp}.27} \ = \ \dfrac{48 \ E \ I_{eq} \ \left( - \ 3 \ L_{1}^{2} \ + \ 4 \ L_{1} \ L_{2}\ + \ L_{2}^{2} \right) }{L_{1}^{3} \ L_{2} \ \left( L_{1} \ + \ L_{2} \right) }
$$

$$ k_{\text{comp}.26} \ = \ k_{\text{comp}.28} \ = \ \dfrac{48 \ E \ I_{eq} \ \left( L_{1}^{2} \ + \ 4 \ L_{1} \ L_{2} \ - \ 3 \ L_{2}^{2} \right) }{L_{1} \ L_{2}^{3} \ \left( L_{1} \ + \ L_{2} \right) } $$

$$ 
k_{\text{comp}.29} \ = \ k_{\text{comp}.30} \ = \ k_{\text{comp}.31} \ = \ k_{\text{comp}.32} \ = \dfrac{72 \ E \ I_{eq}}{L_{1} \ L_{2} \ \left( L_{1} \ + \ L_{2} \right) } $$

In [ ]:
# component 25
k_comp_25_Harada_eq = (48*E*I_Harada_eq*(- 3*L_1_NGM**2 + 4*L_1_NGM*L_2_NGM +   L_2_NGM**2))/(L_1_NGM**3*L_2_NGM*(L_2_NGM + L_1_NGM));

# component 26
k_comp_26_Harada_eq = (48*E*I_Harada_eq*(    L_1_NGM**2 + 4*L_1_NGM*L_2_NGM - 3*L_2_NGM**2))/(L_1_NGM*L_2_NGM**3*(L_2_NGM + L_1_NGM));

# component 29
k_comp_29_Harada_eq = (72*E*I_Harada_eq)/(L_1_NGM*L_2_NGM*(L_2_NGM + L_1_NGM));

# print 
print("The moment of the tube components using Harada EFM-IEq is:\n")
print("components 25 and 27 (face component):", k_comp_25_Harada_eq, "[N/mm].")
print("components 26 and 28 (face component):", k_comp_26_Harada_eq, "[N/mm].")
print("components 29, 30, 31 and 32 (interaction component):", k_comp_29_Harada_eq, "[N/mm].")

---

## Proposed EFM

If the user inputs the same value for the the column width, bC, and the column height, hC (i.e., a square cross-section), the results obtained in this section are for the PS-EFM (both IF and IEq will provide identical results). Otherwise the calculation is performed for PR-EFM, and the PR-EFM-IF and PR-EFM-IEq will provide different results.

### IF: moment of inertia computation

In [ ]:
# length of non-rigid parts of the frame-elements
dim_a = (L_1_NGM - LS) / 2 # face 1 (short)
dim_c = (L_2_NGM - LS) / 2 # face 2 (long)

generic formula:
$$ I_{1} \ = \dfrac{a^3}{24 \ E} \ S_{i.1}  $$

In [ ]:
# moment of inertia of face 1 (short)
I_Proposed_1 = dim_a**3 * Si_NGM_1 / 24 / E
print("The moment of inertia of face 1 (short) using Proposed EFM is:", I_Proposed_1, "[mm^4].")

generic formula:
$$ I_{2}^{\text{PR-EFM}} \ = \ \dfrac{c^3}{24 \ E} \ S_{i.2} $$

In [ ]:
# moment of inertia of face 2 (long)
I_Proposed_2 = dim_c**3 * Si_NGM_2 / 24 / E
print("The moment of inertaia of face 2 (long) using Proposed EFM is:", I_Proposed_2, "[mm^4].")

### IF: stiffness of the tube compoenents

generic formula:
$$ k_{\text{comp}.25} \ = \ k_{\text{comp}.27} \ = \ \dfrac{6 \ E \ I_{1} \ \left( - \ 3 \ I_{2} \ a^{2} \ + \ 4 \ I_{2} \ a \ c\ + \ I_{1} \ c^{2} \right) }{a^{3} \ c \ \left( I_{2} \ a \ + \ I_{1} \ c\right)}
$$ 

$$ k_{\text{comp}.26} \ = \ k_{\text{comp}.28} \ = \ \dfrac{6 \ E \ I_{2} \ \left( I_{2} a^{2} \ + \ 4 \ I_{1} \ a \ c \ - \ 3 \ I_{1} \ c^{2} \right) }{a \ c^{3} \ \left( I_{2} \ a \ + \ I_{1} \ c \right)} $$

$$ k_{\text{comp}.29} \ = \ k_{\text{comp}.30} \ = \ k_{\text{comp}.31} \ = \ k_{\text{comp}.32} \ = \ \dfrac{9 \ E \ I_{1} \ I_{2}}{a \ c \ \left( I_{2} \ a \ + \ I_{1} \ c \right) } $$

In [ ]:
# component 25
k_comp_25_Proposed = (6*E*I_Proposed_1*(- 3*I_Proposed_2*dim_a**2 + 4*I_Proposed_2*dim_a*dim_c +   I_Proposed_1*dim_c**2))/(dim_a**3*dim_c*(I_Proposed_2*dim_a + I_Proposed_1*dim_c));

# component 26
k_comp_26_Proposed = (6*E*I_Proposed_2*(    I_Proposed_2*dim_a**2 + 4*I_Proposed_1*dim_a*dim_c - 3*I_Proposed_1*dim_c**2))/(dim_a*dim_c**3*(I_Proposed_2*dim_a + I_Proposed_1*dim_c));

# component 29
k_comp_29_Proposed = (9*E*I_Proposed_1*I_Proposed_2)/(dim_a*dim_c*(I_Proposed_2*dim_a + I_Proposed_1*dim_c));

# print
print("The moment of the tube components using Proposed EFM-IF is:\n")
print("components 25 and 27 (face component):", k_comp_25_Proposed, "[N/mm].")
print("components 26 and 28 (face component):", k_comp_26_Proposed, "[N/mm].")
print("components 29, 30, 31 and 32 (interaction component):", k_comp_29_Proposed, "[N/mm].")

### IEq: moment of inertia computation

generic formula:
$$I_{eq} = \dfrac{I_{1} + I_{2}}{2}$$

In [ ]:
# equivalent moment of inertia
I_Proposed_eq = (I_Proposed_1 + I_Proposed_2) / 2
print("The equivalent moment of inertia using Proposed EFM is:", I_Proposed_eq, "[mm^4].")

### IEq: stiffness of the tube compoenents

generic formula: 
$$ k_{\text{comp}.25} \ = \ k_{\text{comp}.27} \ = \ \dfrac{6 \ E \ I_{eq} \ \left( - \ 3 \ a^{2} \ + \ 4 \ a \ c \ + \ c^{2}\right) }{a^{3} \ c \ \left( a \ + \ c \right) } $$

$$ k_{\text{comp}.26} \ = \ k_{\text{comp}.28} \ = \ \dfrac{6 \ E \ I_{eq} \ \left( a^{2} \ + \ 4 \ a \ c \ - \ 3 \ c^{2} \right)}{a \ c^{3} \ \left( a \ + \ c \right)} $$

$$ k_{\text{comp}.29} \ = \ k_{\text{comp}.30} \ = \ k_{\text{comp}.31} \ = \ k_{\text{comp}.32} \ = \ \dfrac{9 \ E \ I_{eq}}{a \ c \ \left( a \ + \ c \right)} $$

In [ ]:
# component 25
k_comp_25_Proposed_eq = (6*E*I_Proposed_eq*(- 3*dim_a**2 + 4*dim_a*dim_c +   dim_c**2))/(dim_a**3*dim_c*(dim_a + dim_c));

# component 26
k_comp_26_Proposed_eq = (6*E*I_Proposed_eq*(    dim_a**2 + 4*dim_a*dim_c - 3*dim_c**2))/(dim_a*dim_c**3*(dim_a + dim_c));

# component 29
k_comp_29_Proposed_eq = (9*E*I_Proposed_eq)/(dim_a*dim_c*(dim_a + dim_c));

# print
print("The moment of the tube components using Proposed EFM-IF is:\n")
print("components 25 and 27 (face component):", k_comp_25_Proposed_eq, "[N/mm].")
print("components 26 and 28 (face component):", k_comp_26_Proposed_eq, "[N/mm].")
print("components 29, 30, 31 and 32 (interaction component):", k_comp_29_Proposed_eq, "[N/mm].")

---

## References

For the hyperlinks to work, this section needs to be compiled first.

<a id="ref1"></a>[1] Neves, L.C. and Gomes, F., 1996. Semi-rigid behaviour of beam-to-column minor-axis joints. IABSE reports, pp.207-218.

<a id="ref2"></a>[2] Silva, L.A.P., Neves, L.F.N. and Gomes, F.C.T., 2003. Rotational stiffness of rectangular hollow sections composite joints. Journal of structural engineering, 129(4), pp.487-494.

<a id="ref3"></a>[3] Harada, Y. and da Silva, L.S., 2019. Three-dimensional macro-modeling of beam-to-rectangular hollow section column joints under cyclic loading. Part 1: Modeling of cyclic out-of-plane behavior of single isolated plate element. Journal of Constructional Steel Research, 162, p.105713.

<a id="ref4"></a>[4] Harada, Y. and da Silva, L.S., 2019. Three-dimensional macro-modeling of beam-to-rectangular hollow section column joints under cyclic loading. Part 2: Modeling of beam-to-column joint by extended component-based approach. Journal of Constructional Steel Research, 162, p.105714.

---

## _Notebook Dependencies_

In [ ]:
%load_ext watermark

# python, ipython, packages, and machine characteristics
%watermark -v -m -p pandas,numpy,watermark 

# date
print (" ")
%watermark -u -n -t -z